In [ ]:
#Import the necessary libraries to use Pandas, Numpy, Sklearn, Matplotlib, etc.

import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#AutoGluon

In [ ]:
# Install AutoGluon
!pip install autogluon.tabular

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 285 kB 27.6 MB/s 
     |████████████████████████████████| 59 kB 7.2 MB/s 
     |████████████████████████████████| 224 kB 66.2 MB/s 
     |████████████████████████████████| 280 kB 66.6 MB/s 
     |████████████████████████████████| 1.0 MB 41.1 MB/s 
     |████████████████████████████████| 802 kB 73.2 MB/s 
     |████████████████████████████████| 41 kB 489 kB/s 
     |████████████████████████████████| 132 kB 74.2 MB/s 
     |████████████████████████████████| 79 kB 10.6 MB/s 
     |████████████████████████████████| 9.9 MB 56.2 MB/s 
     |████████████████████████████████| 140 kB 68.1 MB/s 
     |████████████████████████████████| 127 kB 78.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: psutil
    Found existing i

##Cars

In [ ]:
#Define the label column
label = 'price_usd'

In [ ]:
#Load the initial unprocessed data
cars = pd.read_csv('cars.csv')

In [ ]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38531 entries, 0 to 38530
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   manufacturer_name  38531 non-null  object 
 1   model_name         38531 non-null  object 
 2   transmission       38531 non-null  object 
 3   color              38531 non-null  object 
 4   odometer_value     38531 non-null  int64  
 5   year_produced      38531 non-null  int64  
 6   engine_fuel        38531 non-null  object 
 7   engine_has_gas     38531 non-null  bool   
 8   engine_type        38531 non-null  object 
 9   engine_capacity    38521 non-null  float64
 10  body_type          38531 non-null  object 
 11  has_warranty       38531 non-null  bool   
 12  state              38531 non-null  object 
 13  drivetrain         38531 non-null  object 
 14  price_usd          38531 non-null  float64
 15  is_exchangeable    38531 non-null  bool   
 16  number_of_photos   385

In [ ]:
# Split the data to test and train
from sklearn.model_selection import train_test_split
cars_train, cars_test = train_test_split(cars, test_size=0.2, random_state=42)
cars_test_labels = cars_test[label]
cars_test_nolabel = cars_test.drop([label], axis=1)
cars_train_labels = cars_train[label]
cars_train_nolabel = cars_train.drop([label], axis=1)
cars_train.head()

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,price_usd,is_exchangeable,number_of_photos,up_counter,duration_listed
37935,Peugeot,307,mechanical,silver,320000,2005,gasoline,False,gasoline,1.6,universal,False,owned,front,7280.0,False,10.0,1.0,45.5
28568,BMW,525,automatic,violet,400000,1997,diesel,False,diesel,2.5,universal,False,owned,rear,2380.0,True,2.5,0.5,2.5
24311,Chevrolet,Captiva,mechanical,grey,79000,2015,diesel,False,diesel,2.2,suv,False,owned,all,20300.0,False,4.5,6.5,30.5
19393,Jeep,Grand Cherokee,automatic,other,500000,2004,diesel,False,diesel,2.7,suv,False,owned,all,9100.0,False,6.5,39.5,151.0
2540,Opel,Astra,mechanical,silver,280000,2007,diesel,False,diesel,1.7,universal,False,owned,front,7000.0,True,4.0,3.0,33.0


In [ ]:
# Import AutoGluon Libraries
from autogluon.tabular import TabularDataset, TabularPredictor
# Create an instance and then fit the train data
cars_predictor = TabularPredictor(label=label, path='.').fit(cars_train)

Beginning AutoGluon training ...
AutoGluon will save models to "./"
AutoGluon Version:  0.6.0
Python Version:     3.7.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Aug 26 08:44:51 UTC 2022
Train Data Rows:    30824
Train Data Columns: 18
Label Column: price_usd
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (70000.0, 1.4, 9307.82389, 9010.97979)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11972.83 MB
	Train Data (Original)  Memory Usage: 19.15 MB (0.2% of available memory)
	Inferring data type of each feature 

In [ ]:
# Check the performance using test data
cars_pred = cars_predictor.predict(cars_test_nolabel)
cars_perf = cars_predictor.evaluate_predictions(y_true=cars_test_labels, y_pred=cars_pred, auxiliary_metrics=True)

Evaluation: root_mean_squared_error on test data: -2182.490646339327
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -2182.490646339327,
    "mean_squared_error": -4763265.421358653,
    "mean_absolute_error": -1254.1455114519124,
    "r2": 0.940573169985025,
    "pearsonr": 0.9698327836527224,
    "median_absolute_error": -767.8642578125
}


In [ ]:
#Show the leaderboard
cars_predictor.leaderboard(cars_test, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-2182.490646,-2037.466368,1.339245,0.523510,130.201754,0.003452,0.000722,0.188225,2,True,7
1,NeuralNetFastAI,-2216.815179,-2070.624264,0.208097,0.047139,42.548172,0.208097,0.047139,42.548172,1,True,5
2,NeuralNetTorch,-2587.630855,-2489.352977,0.166824,0.061624,55.645325,0.166824,0.061624,55.645325,1,True,6
3,RandomForestMSE,-2728.401229,-2725.610439,0.856681,0.307999,31.785642,0.856681,0.307999,31.785642,1,True,3
4,ExtraTreesMSE,-2779.945778,-2835.280926,1.007290,0.307825,12.035611,1.007290,0.307825,12.035611,1,True,4
5,KNeighborsDist,-6864.161938,-6981.080693,0.106066,0.104261,0.030887,0.106066,0.104261,0.030887,1,True,2
6,KNeighborsUnif,-6920.489506,-6917.648092,0.104191,0.106026,0.034390,0.104191,0.106026,0.034390,1,True,1


In [ ]:
#Show the best model
cars_predictor.get_model_best()

'WeightedEnsemble_L2'

## Cars Downsized by Correlation

In [ ]:
#Check how other columns correlated with the label column
cars_correlation = cars.corr()
cars_correlation['price_usd'].sort_values()

odometer_value     -0.421204
engine_has_gas     -0.062528
is_exchangeable    -0.000503
duration_listed     0.033524
up_counter          0.057382
has_warranty        0.285532
engine_capacity     0.296597
number_of_photos    0.316859
year_produced       0.705511
price_usd           1.000000
Name: price_usd, dtype: float64

In [ ]:
#Drop the columns that do not correlate well with the label column
drop_columns = ['engine_has_gas', 'is_exchangeable', 'duration_listed', 'duration_listed', 'up_counter']
                
cars_downsized = cars.drop(columns = drop_columns)
cars_downsized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38531 entries, 0 to 38530
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   manufacturer_name  38531 non-null  object 
 1   model_name         38531 non-null  object 
 2   transmission       38531 non-null  object 
 3   color              38531 non-null  object 
 4   odometer_value     38531 non-null  int64  
 5   year_produced      38531 non-null  int64  
 6   engine_fuel        38531 non-null  object 
 7   engine_type        38531 non-null  object 
 8   engine_capacity    38521 non-null  float64
 9   body_type          38531 non-null  object 
 10  has_warranty       38531 non-null  bool   
 11  state              38531 non-null  object 
 12  drivetrain         38531 non-null  object 
 13  price_usd          38531 non-null  float64
 14  number_of_photos   38531 non-null  float64
dtypes: bool(1), float64(3), int64(2), object(9)
memory usage: 4.2+ MB


In [ ]:
# Split the data to test and train
from sklearn.model_selection import train_test_split
cars_downsized_train, cars_downsized_test = train_test_split(cars_downsized, test_size=0.2, random_state=42)
cars_downsized_test_labels = cars_downsized_test[label]
cars_downsized_test_nolabel = cars_downsized_test.drop([label], axis=1)
cars_downsized_train_labels = cars_downsized_train[label]
cars_downsized_train_nolabel = cars_downsized_train.drop([label], axis=1)
cars_downsized_train.head()

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,price_usd,number_of_photos
37935,Peugeot,307,mechanical,silver,320000,2005,gasoline,gasoline,1.6,universal,False,owned,front,7280.0,10.0
28568,BMW,525,automatic,violet,400000,1997,diesel,diesel,2.5,universal,False,owned,rear,2380.0,2.5
24311,Chevrolet,Captiva,mechanical,grey,79000,2015,diesel,diesel,2.2,suv,False,owned,all,20300.0,4.5
19393,Jeep,Grand Cherokee,automatic,other,500000,2004,diesel,diesel,2.7,suv,False,owned,all,9100.0,6.5
2540,Opel,Astra,mechanical,silver,280000,2007,diesel,diesel,1.7,universal,False,owned,front,7000.0,4.0


In [ ]:
# Import AutoGluon Libraries
from autogluon.tabular import TabularDataset, TabularPredictor
# Create an instance and then fit the train data
cars_downsized_predictor = TabularPredictor(label=label, path='.').fit(cars_downsized_train)

Beginning AutoGluon training ...
AutoGluon will save models to "./"
AutoGluon Version:  0.6.0
Python Version:     3.7.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Aug 26 08:44:51 UTC 2022
Train Data Rows:    30824
Train Data Columns: 14
Label Column: price_usd
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (70000.0, 1.4, 9307.82389, 9010.97979)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11978.24 MB
	Train Data (Original)  Memory Usage: 18.59 MB (0.2% of available memory)
	Inferring data type of each feature 

In [ ]:
# Get the predictions
cars_downsized_pred = cars_downsized_predictor.predict(cars_downsized_test_nolabel)
cars_downsized_perf = cars_downsized_predictor.evaluate_predictions(y_true=cars_downsized_test_labels, y_pred=cars_downsized_pred, auxiliary_metrics=True)

Evaluation: root_mean_squared_error on test data: -2235.7235470996347
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -2235.7235470996347,
    "mean_squared_error": -4998459.779055772,
    "mean_absolute_error": -1299.1514222901967,
    "r2": 0.9376388688535627,
    "pearsonr": 0.9686898555393884,
    "median_absolute_error": -813.5517578125
}


In [ ]:
#Display the leaderboard
cars_downsized_predictor.leaderboard(cars_downsized_test, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-2235.723547,-2054.520120,1.175637,0.409587,141.724145,0.003037,0.000412,0.171508,2,True,7
1,NeuralNetFastAI,-2265.982800,-2081.780907,0.180167,0.046658,41.104994,0.180167,0.046658,41.104994,1,True,5
2,NeuralNetTorch,-2601.580151,-2460.677424,0.151724,0.054885,78.173399,0.151724,0.054885,78.173399,1,True,6
3,RandomForestMSE,-2721.208913,-2673.882175,0.840708,0.307633,22.274244,0.840708,0.307633,22.274244,1,True,3
4,ExtraTreesMSE,-2730.722072,-2794.785307,0.731850,0.308482,9.721510,0.731850,0.308482,9.721510,1,True,4
5,KNeighborsUnif,-5969.290364,-6059.272219,0.103787,0.106753,0.046864,0.103787,0.106753,0.046864,1,True,1
6,KNeighborsDist,-6005.956842,-6229.222708,0.105805,0.104250,0.033070,0.105805,0.104250,0.033070,1,True,2


In [ ]:
#Show the best model
cars_downsized_predictor.get_model_best()

'WeightedEnsemble_L2'

## Cars Processed

In [ ]:
cars_processed = pd.read_csv('cars_processed.csv')

In [ ]:
from sklearn.model_selection import train_test_split
cars_processed_train, cars_processed_test = train_test_split(cars_processed, test_size=0.2, random_state=42)
cars_processed_test_labels = cars_processed_test[label]
cars_processed_test_nolabel = cars_processed_test.drop([label], axis=1)
cars_processed_train_labels = cars_processed_train[label]
cars_processed_train_nolabel = cars_processed_train.drop([label], axis=1)
cars_processed_train.head()

,Unnamed: 0,age,odometer_value,body_type_suv,number_of_photos,engine_capacity,price_usd
14474,14476,-0.999316,0.228505,0,-0.434623,-0.827156,9380.000
25057,25060,-1.371309,-0.653473,1,-0.270494,0.215801,28700.000
38054,38064,-0.875318,-0.800470,0,-0.762880,-0.678162,12600.000
7880,7880,-0.503324,0.199106,1,-0.762880,0.960770,11900.000
32146,32156,-1.991298,-1.829445,1,0.878406,-0.082187,48695.248


In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
cars_downsized_predictor = TabularPredictor(label=label, path='.').fit(cars_downsized_train)

Beginning AutoGluon training ...
AutoGluon will save models to "./"
AutoGluon Version:  0.6.0
Python Version:     3.7.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Aug 26 08:44:51 UTC 2022
Train Data Rows:    30824
Train Data Columns: 14
Label Column: price_usd
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (70000.0, 1.4, 9307.82389, 9010.97979)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11977.98 MB
	Train Data (Original)  Memory Usage: 18.59 MB (0.2% of available memory)
	Inferring data type of each feature 

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
cars_processed_predictor = TabularPredictor(label=label, path='.').fit(cars_processed_train)

Beginning AutoGluon training ...
AutoGluon will save models to "./"
AutoGluon Version:  0.6.0
Python Version:     3.7.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Aug 26 08:44:51 UTC 2022
Train Data Rows:    30816
Train Data Columns: 6
Label Column: price_usd
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (70000.0, 1.4, 9263.57828, 8971.06939)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11958.64 MB
	Train Data (Original)  Memory Usage: 1.48 MB (0.0% of available memory)
	Inferring data type of each feature ba

In [ ]:
cars_processed_pred = cars_processed_predictor.predict(cars_processed_test_nolabel)
cars_processed_perf = cars_processed_predictor.evaluate_predictions(y_true=cars_processed_test_labels, y_pred=cars_processed_pred, auxiliary_metrics=True)

Evaluation: root_mean_squared_error on test data: -3050.670887412805
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -3050.670887412805,
    "mean_squared_error": -9306592.86330803,
    "mean_absolute_error": -1768.3033216722097,
    "r2": 0.8873817095685564,
    "pearsonr": 0.9423640826751435,
    "median_absolute_error": -1020.425537109375
}


In [ ]:
cars_processed_predictor.leaderboard(cars_processed_test, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-3050.670887,-2861.917566,1.928973,0.740120,207.870625,0.003155,0.000587,0.192079,2,True,7
1,RandomForestMSE,-3085.656176,-2903.732641,0.740980,0.303521,15.922324,0.740980,0.303521,15.922324,1,True,3
2,ExtraTreesMSE,-3149.137327,-2973.593422,0.995598,0.303303,5.907382,0.995598,0.303303,5.907382,1,True,4
3,NeuralNetTorch,-3429.133538,-3229.707758,0.080115,0.029284,185.818675,0.080115,0.029284,185.818675,1,True,6
4,NeuralNetFastAI,-3507.019787,-3347.150034,0.150955,0.038100,29.793605,0.150955,0.038100,29.793605,1,True,5
5,KNeighborsDist,-7481.917812,-7525.547925,0.109125,0.103424,0.030165,0.109125,0.103424,0.030165,1,True,2
6,KNeighborsUnif,-7814.293915,-7855.624750,0.106126,0.104045,0.042724,0.106126,0.104045,0.042724,1,True,1


In [ ]:
cars_processed_predictor.get_model_best()

'WeightedEnsemble_L2'